# Showing samples from Rintracciarinator
Because can't use bokeh on Spyder. Also there are some problems on exporting plots as .png, but one can perform a loop to convert samples dataframes in images.

In [3]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_file, show
from bokeh.models import Range1d
from bokeh.io import output_notebook

In [4]:
i_sample = 120

#df_i_sample = pd.read_csv(r'/home/potruso/Documents/crypto/crypto1 - classic ML/Rintracciarinator/samples/sample_{}.csv'.format(i_sample))
df_i_sample = pd.read_csv(r'/home/potruso/Documents/crypto/crypto1 - classic ML/Rintracciarinator/samples_test/sample_{}.csv'.format(i_sample))

In [5]:
from methods import getHigherHighs

price = np.array(df_i_sample['close'])
dates = df_i_sample.index

K = 2
order = 3
tau = 2*order - 1 # consider the right hh

hh = getHigherHighs(price, order, K, tau)
hh_idx = list(hh[0]) # convert deque to list

# setting coordinates for the segment of the pattern
x_hh = hh_idx
y_hh = df_i_sample['close'].iloc[hh_idx].to_list()

# showing confirmation of local high
hh_idx_conf = list()
for i in hh_idx:
    hh_idx_conf.append(i + order)

x_hh_conf = hh_idx_conf
y_hh_conf = df_i_sample['close'].iloc[hh_idx_conf].to_list()

In [6]:
output_notebook()

Loading BokehJS ...

In [7]:
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(
    title = "Sample #{}".format(i_sample),
    x_axis_type="datetime",
    tools=TOOLS,
    plot_width=700)

p.grid.grid_line_alpha=1 # intensity of grid appearance

inc = df_i_sample['close'] > df_i_sample['open']
dec = df_i_sample['open'] > df_i_sample['close']
w = 0.8 # width

# Plotting candlesticks
p.segment(df_i_sample.index, df_i_sample['high'], df_i_sample.index, df_i_sample['low'], color="black")
p.vbar(df_i_sample.index[inc], w, df_i_sample.open[inc], df_i_sample.close[inc], fill_color="#00af50", line_color="black")
p.vbar(df_i_sample.index[dec], w, df_i_sample.open[dec], df_i_sample.close[dec], fill_color="#F2583E", line_color="black")

# Plotting pattern and confirmation markers
p.line(x_hh, y_hh, color='red', width = 4)
p.scatter(x_hh_conf, y_hh_conf, marker = "circle", size = 10, color = 'black',fill_color = "black")
#p.line([,], [y_hh[-1], y_hh[-1]], color='black', width = 2)

# retracement classes
width = df_i_sample.index[-1] - x_hh[-1] + 1
height_soft = 0.05*y_hh[-1]
height_norm = 0.15*y_hh[-1] - height_soft
height_hard = y_hh[-1] - height_norm - height_soft

p.rect(x = x_hh[-1] + int(width/2.), y = y_hh[-1] - height_soft/2., #setting center
       width = width, height = height_soft,
       color='green', fill_color = "green", alpha = 0.1)
p.rect(x = x_hh[-1] + int(width/2.), y = y_hh[-1] - height_soft - height_norm/2,
       width = width, height = height_norm,
       color='yellow', fill_color = "yellow", alpha = 0.1)
p.rect(x = x_hh[-1] + int(width/2.), y = y_hh[-1] - height_soft - height_norm - height_hard/2,
       width = width, height = height_hard,
       color='orange', fill_color = "orange", alpha = 0.1)

p.y_range = Range1d(min(df_i_sample['low']), max(df_i_sample['high']))
show(p)